In [1]:
import json
from http import HTTPStatus
import dashscope
from dotenv import dotenv_values
from retry import retry
config = dotenv_values('.env')

In [2]:
MODEL_NAME = 'qwen-max'
dashscope.api_key = config['qwen_key'],

@retry(delay=60, tries=3)
def call_qwen_api(MODEL_NAME, query):
    messages = [
        {'role': 'user', 'content': query}]
    response = dashscope.Generation.call(
        MODEL_NAME,
        messages=messages,
        result_format='message',  # set the result is message format.
    )
    if response.status_code == HTTPStatus.OK:
        #print(response)
        return response['output']['choices'][0]['message']['content']
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))
        raise Exception()

In [3]:
def get_prompt(problem, question, options):

    options = '\n'.join(f"{'ABCDEFG'[i]}. {o}" for i, o in enumerate(options))

    prompt = f"""你是一个逻辑推理专家，擅长解决逻辑推理问题。以下是一个逻辑推理的题目，形式为单项选择题。所有的问题都是（close-world assumption）闭世界假设，即未观测事实都为假。请逐步分析问题并在最后一行输出答案，最后一行的格式为"答案是：A"。题目如下：

### 题目:
{problem}

### 问题:
{question}
{options}
"""

    return prompt

In [4]:
def get_question_prompt(data):
    prompt = f"""你是一个逻辑推理问题出题专家，以下JSON是一个problem，对应一些子问题。除了这些子问题外再生成三个其他子问题，保持格式一致并给出参考答案。answer格式为子母序号："A"
### promble如下：
{data}
### 输出格式如下:
{{
promble:
questions:[
question:
options:[]
answer:
]
id:
}}
"""
    return prompt


In [5]:
def read_file(ifn):
    # 读取输入文件
    data = []
    with open(ifn) as reader:
        for line in reader:
            # 解析每一行JSON数据
            sample = json.loads(line)
            data.append(sample)

    return data

In [6]:
def write_jsonl(results, filename):
    with open(f'data/{filename}.json', 'w',encoding='utf-8') as outfile:
        for entry in results:
            json.dump(entry, outfile, ensure_ascii=False)
            outfile.write('\n')
            
def write_json(data, filename):
    with open(f'data/{filename}.json', 'w',encoding='utf-8') as file:
        json.dump(data, file)

In [7]:
import re


def extract_json(response):
    # 使用正则表达式匹配 JSON 数据
    match = re.search(r'```json\s*(.*?)\s*```', response, re.DOTALL)
    if match:
        # 如果找到了 JSON 数据，则返回清理后的 JSON 字符串
        return match.group(1).strip()
    else:
        # 如果没有找到 JSON 数据，则返回原始响应
        return response


def process_data(data):
    query = get_question_prompt(data)
    respond = ''
    try:
        respond = call_qwen_api(MODEL_NAME, query)
        # 清理 JSON 字符串
        respond = extract_json(respond)
        # 解析 JSON 字符串
        data = json.loads(respond)
    except Exception as e:
        #print(respond)
        print(f"Error: {str(e)}")
    return data

In [8]:
file = read_file('data/round1_train_data.jsonl')

In [9]:
from tqdm import tqdm
import concurrent.futures
#file = file[:5]

# 并行批量处理
def batch_process_questions(file, max_workers=5):
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(process_data, data) for data in file]
        results = [future.result() for future in tqdm(concurrent.futures.as_completed(futures), total=len(file))]
    return results

results = batch_process_questions(file)

write_json(results,"output")

  1%|          | 6/500 [01:13<1:28:21, 10.73s/it]

Error: Expecting value: line 6 column 13 (char 400)


  3%|▎         | 13/500 [02:00<58:30,  7.21s/it]  

Error: Expecting value: line 6 column 13 (char 230)


  4%|▍         | 19/500 [02:48<55:48,  6.96s/it]  

...

 96%|█████████▋| 482/500 [1:05:58<02:00,  6.70s/it]

Error: Expecting value: line 6 column 13 (char 594)


 97%|█████████▋| 483/500 [1:06:03<01:43,  6.06s/it]

Error: Expecting value: line 6 column 65 (char 309)


100%|██████████| 500/500 [1:08:37<00:00,  8.23s/it]
